In [1]:
import numpy as np 
import pandas as pd 

In [ ]:
import numpy as np 
import pandas as pd 

!pip install transformers seqeval[gpu]

!pip install simpletransformers

%config Completer.use_jedi = False

import torch

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

data = pd.read_csv('train_ner.csv', encoding='utf-8-sig', sep=';')
data.head()

data.count()

print('number of tags: {}'.format(len(data.Tag.unique())))
freq = data.Tag.value_counts()

data = data.fillna(method = 'ffill')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])
data.head()

data.rename(columns = {'Sentence #':'sentence_id',
                       'Word':'words',
                       'Tag':'label'},inplace = True)

data['label'] = data['label'].str.upper()

In [2]:
!pip install transformers seqeval[gpu]

In [3]:
!pip install simpletransformers

In [4]:
%config Completer.use_jedi = False

In [5]:
from sklearn.metrics import accuracy_score
import torch

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
data = pd.read_csv('train_ner.csv', encoding='utf-8-sig', sep=';')
data.head()

,Sentence #,Word,POS,Tag
0,1.0,На,PR,O
1,NaN,севере,S,O
2,NaN,граничит,V,O
3,NaN,с,PR,O
4,NaN,Латвией,S,I-LOC


In [8]:
data.count()

Sentence #      63477
Word          1043155
POS           1047002
Tag           1046207
dtype: int64

In [9]:
print('number of tags: {}'.format(len(data.Tag.unique())))
freq = data.Tag.value_counts()
freq

number of tags: 10


O         868357
I-LOC      67787
I-MISC     51002
I-PER      34921
I-ORG      23710
B-LOC        254
B-MISC       127
B-PER         33
B-ORG         16
Name: Tag, dtype: int64

In [10]:
data = data.fillna(method = 'ffill')

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])
data.head()

,Sentence #,Word,POS,Tag
0,0,На,PR,O
1,0,севере,S,O
2,0,граничит,V,O
3,0,с,PR,O
4,0,Латвией,S,I-LOC


In [13]:
data.rename(columns = {'Sentence #':'sentence_id',
                       'Word':'words',
                       'Tag':'label'},inplace = True)

In [14]:
data

,sentence_id,words,POS,label
0,0,На,PR,O
1,0,севере,S,O
2,0,граничит,V,O
3,0,с,PR,O
4,0,Латвией,S,I-LOC
...,...,...,...,...
1048531,63455,корпуса,S,O
1048532,63455,.,SENT,O
1048533,63456,В,PR,O
1048534,63456,России,S,I-LOC


In [15]:
data['label'] = data['label'].str.upper()

In [16]:
X = data[['sentence_id','words']]
y = data['label']

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [18]:
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [19]:
len(train_data)

786402

In [20]:
!pip install pyarrow

In [21]:
from simpletransformers.ner import NERModel, NERArgs

In [22]:
labels = data['label'].unique().tolist()
labels

['O', 'I-LOC', 'B-LOC', 'I-ORG', 'I-MISC', 'I-PER', 'B-PER', 'B-MISC', 'B-ORG']

In [23]:
args = NERArgs()
args.num_train_epochs = 20
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [24]:
model = NERModel('bert','DeepPavlov/rubert-base-cased',labels = labels , args = args)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [ ]:
model.train_model(train_data , eval_data = test_data , acc = accuracy_score)

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/1983 [00:00<?, ?it/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/simpletransformers/ner/ner_model.py:810: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/srv/conda/envs/saturn/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 20:   0%|          | 0/1983 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
result

In [ ]:
model.save_model('NER_DeepPavlov_rubert-base-cased_1')

In [ ]:
new_data = pd.read_csv("workspace/Train_Only_Sentence/Train_Only_Sentence_9.csv", encoding='utf-8-sig', sep=';')
new_data.head()

In [ ]:
import csv

In [ ]:
negative = []

for cur in new_data['text']:
  negative.append(cur)

print(len(negative))

In [ ]:
print(negative[0])

In [ ]:
prediction, model_output = model.predict(negative)

In [ ]:
df = pd.DataFrame({'ner': prediction})

In [ ]:
df.to_csv('ner_predictions_9.csv', index=False)